In [0]:
# %pip list

In [0]:
# %edit /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras_vggface/models.py

In [0]:
# with open('/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras_vggface/models.py', 'r') as file:
#     content = file.read()

# new_content = content.replace('from keras.engine.topology import get_source_inputs', 'from keras.utils import get_source_inputs')

# with open('/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras_vggface/models.py', 'w') as file:
#     file.write(new_content)

In [0]:
# with open('/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras_vggface/models.py', 'r') as file:
#     for line in file:
#         print(line)
# #     content = file.read()
# # content

'''VGGFace models for Keras.



# Notes:

- Resnet50 and VGG16  are modified architectures from Keras Application folder. [Keras](https://keras.io)



- Squeeze and excitation block is taken from  [Squeeze and Excitation Networks in

 Keras](https://github.com/titu1994/keras-squeeze-excite-network) and modified.



'''





from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D, \

    GlobalMaxPooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, \

    AveragePooling2D, Reshape, Permute, multiply

from keras_applications.imagenet_utils import _obtain_input_shape

from keras.utils import layer_utils

from keras.utils.data_utils import get_file

from keras import backend as K

from keras_vggface import utils

from keras.utils import get_source_inputs

import warnings

from keras.models import Model

from keras import layers





def VGG16(include_top=True, weights='vggface',

          input_tensor=None, input_shape=None,

          pooling=None,

          c

In [0]:
import numpy as np
import pandas as pd
import os
import errno
import imageio
import time
import os
import cv2
from mtcnn.mtcnn import MTCNN

         
      # from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import ImageDataGenerator

        # from tensorflow.python.keras import models
from keras import models
        # from tensorflow.keras import models
        # from tensorflow.python.keras import layers
from keras import layers
        # from tensorflow.keras import layers
        # from tensorflow.keras import layers
# from tensorflow.python.keras import Model
from keras import Model
      # from tensorflow.keras import Model
# from keras.optimizers import SGD
        # from tensorflow.keras.optimizers import SGD
# from keras.optimizers.legacy import SGD
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

from keras_vggface.vggface import VGGFace
      #from keras.utils import get_source_inputs

In [0]:


# initialization
image_height = 48
image_width = 48
window_size = 24
window_step = 6
SAVE_IMAGES = True
SELECTED_LABELS = [0,1,2,3,4,5,6]
IMAGES_PER_LABEL = 500
OUTPUT_FOLDER_NAME = "/FileStore/demo_store"

def data_download():
    print( "preparing")
    try:
        os.makedirs(OUTPUT_FOLDER_NAME)
    except OSError as e:
        if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
            pass
        else:
            raise
            
    
    print( "importing csv file")
    
    # Load the dataset
    data_set_path = "/FileStore/demo_data/fer2013.csv"
    data = spark.read.csv(data_set_path,header=True,inferSchema=True)
    # display(data)
    data = data.toPandas()

    # data = pd.read_csv('../datasets/raw/fer2013.csv') # data from kaggle stored here as csv file.
    print(data.head(2))

    for category in data['Usage'].unique():
        # print(category)
        # time.sleep(3)
        print( "converting set: " + category + "...")
        # create folder
        if not os.path.exists(category):
            try:
                os.makedirs(OUTPUT_FOLDER_NAME + '/' + category)
            except OSError as e:
                if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
                   pass
                else:
                    raise
        
        # get samples and labels of the actual category
        category_data = data[data['Usage'] == category]
        samples = category_data['pixels'].values
        labels = category_data['emotion'].values
        
        # print("\ncategory:",category_data,"\nsamples:",samples[0],"\nlabels:",labels)
        # time.sleep(3)
        # get images and extract features
        images = []
   
        print("Length of samples,labels\n:::::")
        print(len(samples),len(labels))
        print("\nSelected labels:",SELECTED_LABELS)
        # print(type(labels),type(samples))
        # time.sleep(2)
        for i in range(len(samples)):
            # print("inside for loop:",i)
            try:
                if labels[i] in SELECTED_LABELS: 
                    if i%1000 == 0:
                        print(i,"inside if class:")
                    # image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
                    image = np.fromstring(samples[i], dtype=np.uint8, sep=" ").reshape((image_height, image_width))
                    images.append(image)
                    # print(image)
                    imageio.imwrite(OUTPUT_FOLDER_NAME + '/' + category + '/' + str(i) + '.jpg', image)
                    # break
                    
            except Exception as e:
                print( "error in image: " + str(i) + " - " + str(e))
                break
        print("image count",len(images))        
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/images.npy', images)


In [0]:
import itertools

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def data_generator(train_dir, valid_dir, test_dir, train_label, valid_label, test_label):
    # Add our data-augmentation parameters to ImageDataGenerator
    train_datagen = ImageDataGenerator(#rescale = 1./255.,
                                       rotation_range = 40,
                                       width_shift_range = 0.2,
                                       height_shift_range = 0.2,
                                       shear_range = 0.2,
                                       zoom_range = 0.2,
                                       horizontal_flip = True)
    
    # Note that the validation data should not be augmented!
    valid_datagen = ImageDataGenerator()#rescale = 1./255. )
    test_datagen = ImageDataGenerator()#rescale = 1./255. )
    
    # Flow training images in batches of 5 using train_datagen generator
    train_generator=train_datagen.flow_from_dataframe(dataframe=train_label,
                                                     directory=train_dir,
                                                     x_col="id",
                                                     y_col="emotion",
                                                     target_size=(96,96),
                                                     batch_size=32,
                                                     seed=42,
                                                     shuffle=True,
                                                     class_mode="categorical",
                                                     color_mode='rgb')
    # train_generator=itertools.islice(itertools.cycle(train_generator),896)#By using itertools.cycle() and itertools.islice(), you can achieve a similar effect to the .repeat()
    # train_generator=train_generator.repeat() // I deleted steps_per_epoch

    
    # # Flow validation images in batches of 5 using test_datagen generator
    valid_generator=valid_datagen.flow_from_dataframe(dataframe=valid_label,
                                                      directory=valid_dir,
                                                      x_col="id",
                                                      y_col="emotion",
                                                      target_size=(96,96),
                                                      batch_size=32,
                                                      seed=42,
                                                      shuffle=True,
                                                      class_mode="categorical",
                                                      color_mode='rgb')
    
    # valid_generator=itertools.islice(itertools.cycle(valid_generator),896)#By using itertools.cycle() and itertools.islice(), you can achieve a similar effect to the .repeat()
    # valid_generator=valid_generator.repeat() I deleted steps_per_valid
    # Flow validation images in batches of 5 using test_datagen generator
    test_generator=test_datagen.flow_from_dataframe(dataframe=test_label,
                                                     directory=test_dir,
                                                     x_col="id",
                                                     y_col=None,
                                                     target_size=(96,96),
                                                     batch_size=32,
                                                     seed=42,
                                                     shuffle=False,
                                                     class_mode=None,
                                                     color_mode='rgb')

    return train_generator, valid_generator, test_generator


In [0]:
# -*- coding: utf-8 -*-
"""
Created on 30 May 2023

Author: Bibin Kunjumon
"""

def append_ext(fn):
    return fn+".jpg"

In [0]:
# -*- coding: utf-8 -*-
"""
Created on 30 May 2023
File: label_and_dir.py
Author: Bibin Kunjumon

"""

def label_and_dir():
    # Load the dataset
    data_set_path = "/FileStore/demo_store/traininglabel.csv"
    train_label_spark = spark.read.csv(data_set_path,header=True,inferSchema=True)
    train_label = train_label_spark.toPandas().astype(str)

    data_set_path = "/FileStore/demo_store/publictestlabel.csv"
    valid_label_spark = spark.read.csv(data_set_path,header=True,inferSchema=True)
    valid_label = valid_label_spark.toPandas().astype(str)

    data_set_path = "/FileStore/demo_store/privatetestlabel.csv"
    test_label_spark = spark.read.csv(data_set_path,header=True,inferSchema=True)
    test_label = test_label_spark.toPandas().astype(str)


    # train_label = pd.read_csv('/FileStore/demo_store/traininglabel.csv',dtype=str)
    # valid_label = pd.read_csv('/FileStore/demo_store/publictestlabel.csv',dtype=str)
    # test_label = pd.read_csv('/FileStore/demo_store/privatetestlabel.csv',dtype=str)
    
    train_label["id"]=train_label["id"].apply(append_ext)
    valid_label["id"]=valid_label["id"].apply(append_ext)
    test_label["id"]=test_label["id"].apply(append_ext)
    
    # Define our example directories and files
    train_dir = '/FileStore/demo_store/Training'
    valid_dir = '/FileStore/demo_store/PublicTest'
    test_dir = '/FileStore/demo_store/PrivateTest'
    return train_dir, valid_dir, test_dir, train_label, valid_label, test_label

In [0]:

#downloading data from Kaggle
data_download()

#importing data
train_dir, valid_dir, test_dir, train_label, valid_label, test_label = label_and_dir()
train_generator, valid_generator, test_generator = data_generator(train_dir, valid_dir, test_dir, train_label, valid_label, test_label)

#custom parameters
nb_class = 7
hidden_dim = 1024

preparing
importing csv file
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
converting set: Training...
Length of samples,labels
:::::
28709 28709

Selected labels: [0, 1, 2, 3, 4, 5, 6]
0 inside if class:
1000 inside if class:
2000 inside if class:
3000 inside if class:
4000 inside if class:
5000 inside if class:
6000 inside if class:
7000 inside if class:
8000 inside if class:
9000 inside if class:
10000 inside if class:
11000 inside if class:
12000 inside if class:
13000 inside if class:
14000 inside if class:
15000 inside if class:
16000 inside if class:
17000 inside if class:
18000 inside if class:
19000 inside if class:
20000 inside if class:
21000 inside if class:
22000 inside if class:
23000 inside if class:
24000 inside if class:
25000 inside if class:
26000 inside if class:
27000 inside if class:
28000 inside 

In [0]:
vgg_model = VGGFace(include_top=False, input_shape=(96, 96, 3))
last_layer = vgg_model.get_layer('pool5').output
print(last_layer.shape)

(None, 3, 3, 512)


In [0]:
layers.Flatten()(last_layer)

<KerasTensor: shape=(None, 4608) dtype=float32 (created by layer 'flatten_2')>

In [0]:
x = layers.Flatten()(last_layer)
x = layers.Dense(hidden_dim, activation='relu', name='fc7')(x)
x = layers.Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, x)

print("---Printing the model summary\n")
#Printing the model summary
custom_vgg_model.summary()

---Printing the model summary

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 96, 96, 64)        1792      
                                                                 
 conv1_2 (Conv2D)            (None, 96, 96, 64)        36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 48, 48, 64)        0         
                                                                 
 conv2_1 (Conv2D)            (None, 48, 48, 128)       73856     
                                                                 
 conv2_2 (Conv2D)            (None, 48, 48, 128)       147584    
                                                                 
 pool2 (MaxPooling2D)       

In [0]:
# Training the model with fer2013 data.
custom_vgg_model.compile(loss='categorical_crossentropy',
              optimizer=SGD(learning_rate=0.0001),
              metrics=['accuracy'])
print("vgg_model compiled")

vgg_model compiled


In [0]:
#Creating a callback that saves a model when the validation loss decreases from the previous epoch.
filepath="/FileStore/demo_store/epoch_store/weights-improvement-{epoch:02d}-{val_loss:.3f}"
checkpoint = ModelCheckpoint(filepath, monitor ='val_loss', verbose=1, save_best_only=False, mode='auto', save_weights_only = False)
callbacks = [checkpoint]

# dbutils.fs.put("/FileStore/demo_store/trained_models/")

In [0]:
import urllib3

http = urllib3.PoolManager(maxsize=10)

In [0]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [0]:

print("Starting the model.fit: each Epoch takes time ")
history = custom_vgg_model.fit(
    train_generator,
    validation_data=valid_generator,validation_split=0.1,
    # steps_per_epoch=897, #TensorFlow will automatically determine it based on the number of available batches in the training generator.
    epochs=30,
    # validation_steps=897,
    verbose=1,
    callbacks=callbacks,
)

In [0]:
custom_vgg_model.save("/FileStore/demo_store/trained_models/emotion_trained_june")

INFO:tensorflow:Assets written to: /FileStore/demo_store/trained_models/emotion_trained_june/assets


INFO:tensorflow:Assets written to: /FileStore/demo_store/trained_models/emotion_trained_june/assets


In [0]:
# %rm /FileStore/demo_store/trained_models/*

In [0]:
from keras.models import load_model

In [0]:

my_model = load_model("/FileStore/demo_store/trained_models/emotion_trained_june")
# my_model = load_model("/dbfs/FileStore/demo_store/built_model/trained_vggface.h5")

In [0]:
my_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 96, 96, 64)        1792      
                                                                 
 conv1_2 (Conv2D)            (None, 96, 96, 64)        36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 48, 48, 64)        0         
                                                                 
 conv2_1 (Conv2D)            (None, 48, 48, 128)       73856     
                                                                 
 conv2_2 (Conv2D)            (None, 48, 48, 128)       147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 24, 24, 128)       0   